In [1]:
#
# This program defines a grid of points on the surface of a geoid.
# From a given point (P0) a number of "columns" and "rows" are defined.
# The size of the grid is specified by introducing:
# - the number of columns to the "right" and "left" of P0, 
# - the number of rows "below" and "above" P0.
# The points are created so that the geographic distance between them (row-row and col-col) is a given value.
# All points in the same row than P0 have the same latitude as P0.
# All points in the same column than P0 share with it the longitude.
# The distances are calculated with a geodesic formula.
# Units: 
# - distance: km.
# - coordinates: decimal degrees.
# The coordinates are used as (lon, lat) NOT as (lat, lon).
# The results are exporte as:
# - a csv file with the coordinates of the points,
# - a geojson file with the polygons that can be defined from each 4-set of adjacent points.


In [2]:
# Version log.
# 
# R0 (20200401)
# All parts tested and put together.
# Seems to work well.


In [3]:
# Files and directories.

# Directories:
RootDir = 'D:/0 DOWN/00 PY RG/GIS/GRIDS/'

# I/O files:
FileNameOut1 = RootDir + 'Data_out 06 pnt R0.csv'
FileNameOut2 = RootDir + 'Data_out 06 pol R0.geojson'


In [4]:
# Import modules.
from math import radians, degrees, cos, copysign
from geopy.distance import geodesic, lonlat
import csv, json


In [5]:
# Functions.
def f_dlon(lat, dist, R = 6371):
    """
    Calculate the increase in longitude required to cover a given distance along a parallel.
    Asumes a sphere.
    Data in degrees and km.
    """
    return degrees(dist / (R * cos(radians(lat))))

def f_dlat(dist, R = 6371):
    """
    Calculate the increase in latitude required to cover a given distance along a meridian.
    Asumes a sphere.
    Data in degrees and km.
    """
    return degrees(dist / R)


In [6]:
# Input data.
# Starting point:
P0 = [6., 45.]
# Geographic distance between points of the grid (x and y-wise):
D0 = 100.
# Grid size, measured as number or rows/columns :
nx_1 = 16 # decreasing longitude, columns - excluding the reference 
nx_2 = 21 # increasing longitude, columns - excluding the reference
ny_1 = 11 # decreasing latitude, rows - excluding the reference
ny_2 = 18 # increasing latitude, rows - excluding the reference

# Iteration parameters:
# Accepted absolute error:
abs_err = D0 / 1000.
# Maximum number of iterations:
iter_max = 1000

# Associated variables:
# Grid size (total number of rows/columns):
NX = nx_1 + nx_2 + 1 # longitude
NY = ny_1 + ny_2 + 1 # latitude


In [7]:
# Check suitability of the data.
# For extreme latitudes:
lat_min, lat_max = -80., 80. # Arbitrarily close to problematic latitudes
lat_1 = P0[1] - f_dlat((ny_1)*D0)
lat_2 = P0[1] + f_dlat((ny_2)*D0)
if (lat_1 < lat_min) or (lat_2 > lat_max):
    print (' lat_min = {0:4.2f}, lat_max = {1:4.2f}'.format(lat_1, lat_2))
    raise Exception (' Warning: extreme latitudes exceeded. Program stopped.')

# For extreme longitudes:
lat_aux = max(abs(lat_1), abs(lat_2))
lon_aux = f_dlon(lat_aux, (NX-1)*D0)
lon_max = 180. # Arbitrarily chosen.
if (lon_aux > lon_max):
    print (' lon_max = {0:4.2f}'.format(lon_aux))
    raise Exception (' Warning: extreme longitude exceeded. Program stopped.')


In [8]:
# Prepare the container of the points, a list of lists.
aux = float('nan')

# The access is first from x and then from y!
data = []
for j in range (0, NX, 1):
    aux_j = []
    for i in range (0, NY, 1):
        aux_j.append([aux, aux])
    data.append(aux_j)
    
# Enter P00:
data[nx_1 ][ny_1] = P0


In [9]:
# Generate the central row, at the middle of the grid.
# Right side:
for x in range(nx_1, NX-1, 1):
    # P1 is defined:
    P1      = data[x][ny_1]
    # P2 is on the same parallel than P1:
    P2      = [P1[0] + f_dlon(P1[1], D0), P1[1]]
    D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
    D12_err = D12 - D0
    iter_count   = 0
    while (abs(D12_err) > abs_err) and (iter_count <= iter_max):
        lon_aux = f_dlon(P2[1], abs(D12_err/2))
        P2      = [P2[0] - copysign(lon_aux, D12_err), P2[1]]
        D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
        D12_err = D12 - D0
        iter_count  += 1
        if (iter_count == iter_max):
            print (" Warning at P2: iter_max exceeded. D_err = ", abs(D12_err)) 

    # Save to data_list:
    data[x+1][ny_1] = P2

# Left side:
for x in range(nx_1, 0, -1):
    # P2 is defined:
    P2      = data[x][ny_1]
    # P1 is on the same parallel than P2:
    P1      = [P2[0] - f_dlon(P2[1], D0), P2[1]]
    D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
    D12_err = D12 - D0
    iter_count   = 0
    while (abs(D12_err) > abs_err) and (iter_count < iter_max):
        lon_aux = f_dlon(P1[1], abs(D12_err/2))
        P1      = [P1[0] + copysign(lon_aux, D12_err), P1[1]]
        D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
        D12_err = D12 - D0
        iter_count  += 1
        if (iter_count == iter_max):
            print (" Warning at P1: iter_max exceeded. D_err = ", abs(D12_err))

    # Save to data_list:
    data[x-1][ny_1] = P1


In [10]:
# Generate the central column, at the center of the grid.
# Upper side:
for y in range(ny_1, NY-1, 1):
    # P1 is defined:
    P1      = data[nx_1][y]
    # P4 is on the same meridian than P1:
    P4      = [P1[0], P1[1] + f_dlat(D0)]
    D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
    D14_err = D14 - D0
    iter_count   = 0
    while (abs(D14_err) > abs_err) and (iter_count <= iter_max):
        lat_aux = f_dlat(abs(D14_err/2))
        P4      = [P4[0], P4[1] - copysign(lat_aux, D14_err)]
        D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
        D14_err = D14 - D0
        iter_count  += 1
        if (iter_count == iter_max):
            print (" Warning at P4: iter_max exceeded. D_err = ", abs(D14_err))
    
    # Save to data_list:
    data[nx_1][y+1] = P4

# Lower side:
for y in range(ny_1, 0, -1):
    # P4 is defined:
    P4      = data[nx_1][y]
    # P1 is on the same meridian than P4:
    P1      = [P4[0], P4[1] - f_dlat(D0)]
    D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
    D14_err = D14 - D0
    iter_count   = 0
    while (abs(D14_err) > abs_err) and (iter_count <= iter_max):
        lat_aux = f_dlat(abs(D14_err/2))
        P1      = [P1[0], P1[1] + copysign(lat_aux, D14_err)]
        D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
        D14_err = D14 - D0
        iter_count  += 1
        if (iter_count == iter_max):
            print (" Warning at P1: iter_max exceeded. D_err = ", abs(D14_err))
    
    # Save to data_list:
    data[nx_1][y-1] = P1


In [11]:
# Complete the grid from the central row and column.
# First quadrant:
for x in range(nx_1, NX-1, 1):
    for y in range(ny_1, NY-1, 1):
        # P1, P2 and P4 are defined:
        P1 = data[x  ][y  ]
        P2 = data[x+1][y  ]
        P4 = data[x  ][y+1]
        # P3 is approx. on the same longitude that P2, and approx. on the same latitude than P4:
        P3      = [P2[0], P4[1]]
        D23     = geodesic(lonlat(*P2), lonlat(*P3)).km
        D34     = geodesic(lonlat(*P3), lonlat(*P4)).km
        D23_err = D23 - D0
        D34_err = D34 - D0
        D_err   = max(abs(D23_err), abs(D34_err))
        iter_count   = 0
        while (abs(D_err) > abs_err) and (iter_count <= iter_max):
            lat_aux = f_dlat(abs(D23_err/2))
            lon_aux = f_dlon(P3[1] - copysign(lat_aux, D23_err), abs(D34_err/2))
            P3      = [P3[0] - copysign(lon_aux, D34_err), P3[1] - copysign(lat_aux, D23_err)]
            D23     = geodesic(lonlat(*P2), lonlat(*P3)).km
            D34     = geodesic(lonlat(*P3), lonlat(*P4)).km
            D23_err = D23 - D0
            D34_err = D34 - D0
            D_err   = max(abs(D23_err), abs(D34_err))
            iter_count  += 1
            if (iter_count == iter_max):
                print (" Warning at P3: iter_max exceeded. D_err = ", D_err) 
    
        # Save to data_list:
        data[x+1][y+1] = P3


In [12]:
# Second quadrant:
for x in range(nx_1, 0, -1):
    for y in range(ny_1, NY-1, 1):
        # P1, P2 and P3 are defined:
        P1 = data[x-1][y  ]
        P2 = data[x  ][y  ]
        P3 = data[x  ][y+1]
        # P4 is approx. on the same longitude that P1, and approx. on the same latitude than P3:
        P4      = [P1[0], P3[1]]
        D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
        D34     = geodesic(lonlat(*P3), lonlat(*P4)).km
        D14_err = D14 - D0
        D34_err = D34 - D0
        D_err   = max(abs(D14_err), abs(D34_err))
        iter_count   = 0
        while (abs(D_err) > abs_err) and (iter_count <= iter_max):
            lat_aux = f_dlat(abs(D14_err/2))
            lon_aux = f_dlon(P4[1] - copysign(lat_aux, D14_err), abs(D34_err/2))
            P4      = [P4[0] + copysign(lon_aux, D34_err), P4[1] - copysign(lat_aux, D14_err)]
            D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
            D34     = geodesic(lonlat(*P3), lonlat(*P4)).km
            D14_err = D14 - D0
            D34_err = D34 - D0
            D_err   = max(abs(D14_err), abs(D34_err))
            iter_count  += 1
            if (iter_count == iter_max):
                print (" Warning at P4: iter_max exceeded. D_err = ", D_err) 
    
        # Save to data_list:
        data[x-1][y+1] = P4

In [13]:
# Third quadrant:
for x in range(nx_1, 0, -1):
    for y in range(ny_1, 0, -1):
        # P2, P3 and P4 are defined:
        P2 = data[x  ][y-1]
        P3 = data[x  ][y  ]
        P4 = data[x-1][y  ]
        # P1 is approx. on the same longitude that P4, and approx. on the same latitude than P2:
        P1      = [P4[0], P2[1]]
        D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
        D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
        D12_err = D12 - D0
        D14_err = D14 - D0
        D_err   = max(abs(D12_err), abs(D14_err))
        iter_count   = 0
        while (abs(D_err) > abs_err) and (iter_count <= iter_max):
            lat_aux = f_dlat(abs(D14_err/2))
            lon_aux = f_dlon(P1[1] + copysign(lat_aux, D14_err), abs(D12_err/2))
            P1      = [P1[0] + copysign(lon_aux, D12_err), P1[1] + copysign(lat_aux, D14_err)]
            D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
            D14     = geodesic(lonlat(*P1), lonlat(*P4)).km
            D12_err = D12 - D0
            D14_err = D14 - D0
            D_err   = max(abs(D12_err), abs(D14_err))
            iter_count  += 1
            if (iter_count == iter_max):
                print (" Warning at P1: iter_max exceeded. D_err = ", D_err) 
    
        # Save to data_list:
        data[x-1][y-1] = P1


In [14]:
# Fourth quadrant:
for x in range(nx_1, NX-1, 1):
    for y in range(ny_1, 0, -1):
        # P1, P3 and P4 are defined:
        P1 = data[x  ][y-1]
        P3 = data[x+1][y  ]
        P4 = data[x  ][y  ]
        # P2 is approx. on the same longitude that P3, and approx. on the same latitude than P1:
        P2      = [P3[0], P1[1]]
        D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
        D23     = geodesic(lonlat(*P2), lonlat(*P3)).km
        D12_err = D12 - D0
        D23_err = D23 - D0
        D_err   = max(abs(D12_err), abs(D23_err))
        iter_count   = 0
        while (abs(D_err) > abs_err) and (iter_count <= iter_max):
            lat_aux = f_dlat(abs(D23_err/2))
            lon_aux = f_dlon(P2[1] + copysign(lat_aux, D23_err), abs(D12_err/2))
            P2      = [P2[0] - copysign(lon_aux, D12_err), P2[1] + copysign(lat_aux, D23_err)]
            D12     = geodesic(lonlat(*P1), lonlat(*P2)).km
            D23     = geodesic(lonlat(*P2), lonlat(*P3)).km
            D12_err = D12 - D0
            D23_err = D14 - D0
            D_err   = max(abs(D12_err), abs(D23_err))
            iter_count  += 1
            if (iter_count == iter_max):
                print (" Warning at P2: iter_max exceeded. D_err = ", D_err) 
                print (x, y)
    
        # Save to data_list:
        data[x+1][y-1] = P2


In [ ]:
# Check distances.
# Vertical:
for x in range(0, NX, 1):
    for y in range(0, NY-1, 1):
        P1 = data[x][y  ]
        P4 = data[x][y+1]
        print(geodesic(lonlat(*P1), lonlat(*P4)))

# Horizontal:
for y in range(0, NY, 1):
    for x in range(0, NX-1, 1):
        P1 = data[x  ][y]
        P2 = data[x+1][y]
        print(geodesic(lonlat(*P1), lonlat(*P2)))


In [17]:
# Save the grid.
# Points:
with open(FileNameOut1, 'w', newline = '') as fileCSVout:
    writeCSV = csv.writer(fileCSVout, delimiter = ',')
    for x in range(0, NX, 1):
        writeCSV.writerows(data[x])


In [18]:
# Create the dictionary for the geojson FeatureCollection.
fc = {"type": "FeatureCollection", "features": []}

# Add the polygons:
i=0
for x in range(0, NX-1, 1):
    for y in range(0, NY-1, 1):
        aux = [(p) for p in [data[x][y], data[x+1][y], data[x+1][y+1], data[x][y+1]]]
        f ={"type": "Feature", 
            "geometry": {"type": "Polygon", "coordinates": [aux]},
            "properties": {"pol": "pol_from_grid", "id": i}
            }
        fc["features"].append(f)
        i += 1


In [19]:
# Save as geojson.
# json conversion is needed because the python dictionary uses '' while geojson requires "".
with open(FileNameOut2, 'w') as output_file:
    output_file.write(json.dumps(fc))
